In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
# re-size all the images to this
IMAGE_SIZE = [299, 299]

train_path = '../input/car-categories-dataset/New Car Dataset/train'
valid_path = '../input/car-categories-dataset/New Car Dataset/validation'
test_path = '../input/car-categories-dataset/New Car Dataset/test'


In [4]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inceptionresnetv2 = InceptionResNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [5]:
# don't train existing weights
for layer in inceptionresnetv2.layers:
    layer.trainable = False

In [6]:
  # useful for getting number of output classes
folders = glob('../input/car-categories-dataset/New Car Dataset/train/*')

In [7]:
# our layers - you can add more if you want
x = Flatten()(inceptionresnetv2.output)
#x = Dense(units=8, activation='relu')(x)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inceptionresnetv2.input, outputs=prediction)

In [9]:

# view the structure of the model
model.summary()


In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('../input/car-categories-dataset/New Car Dataset/train',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
valid_set = test_datagen.flow_from_directory('../input/car-categories-dataset/New Car Dataset/validation',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('../input/car-categories-dataset/New Car Dataset/test',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
print(len(training_set))
print(len(valid_set))
print(len(test_set))

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=valid_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(valid_set)
)

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points


In [ ]:
smooth_val_acc_history = smooth_curve(r.history['val_accuracy'][:])
smooth_train_acc_history = smooth_curve(r.history['accuracy'][:])

fig = plt.figure(figsize=(15,5))
ax = fig.subplots(1, 2)
ax[0].plot(range(1, len(smooth_val_acc_history) + 1), smooth_val_acc_history)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Validation Accuracy')

ax[1].plot(range(1, len(smooth_train_acc_history) + 1), smooth_train_acc_history)
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Training Accuracy')

plt.show()

In [ ]:
model.evaluate(test_set)

In [ ]:
y_hat = model.predict(test_set)

In [ ]:
y_hat

In [ ]:
test_set.class_indices

In [ ]:
test_set.labels

In [ ]:
y_hat = np.argmax(model.predict(test_set), axis=1)
y_hat

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 10))


ax1 = sns.distplot(test_set.labels, hist=False, color="r", label="Actual Value")
sns.distplot(y_hat, hist=False, color="b", label="Fitted Values" , ax=ax1)


plt.title('Actual vs Fitted Category')
plt.xlabel('Categories')
plt.ylabel('Proportion of Classifications')

plt.show()
plt.close()

In [ ]:
# save it as a h5 file
from tensorflow.keras.models import load_model

model.save('./model_inceptionresnetv2.h5')

<a href="./model_inceptionresnetv2.h5"> Download Model </a>

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.preprocessing import image

In [ ]:
model = load_model('./model_inceptionresnetv2.h5')
save_model(model, './my_model')

In [ ]:
img=image.load_img('../input/car-categories-dataset/New Car Dataset/test/large/10106.jpg',target_size=(299,299))

In [ ]:
x=image.img_to_array(img)
x

In [ ]:
x.shape

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
plt.imshow(img)

In [ ]:
img=image.load_img('../input/car-categories-dataset/New Car Dataset/test/small/10235.jpg',target_size=(299,299))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
model.predict(img_data)

In [ ]:
plt.imshow(img)

In [ ]:
img=image.load_img('../input/car-categories-dataset/New Car Dataset/test/medium/13105.jpg',target_size=(299,299))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
model.predict(img_data)

In [ ]:
plt.imshow(img)